In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image
from tqdm import tqdm
from scipy import stats

In [2]:
device = torch.device('cuda:0')

In [3]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [4]:
class ImageDataset(Dataset):
    def __init__(self, paths, labels, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img_name = self.paths[idx]
        img = Image.open(img_name).convert('RGB')
        
        img_tensor = self.transform(img)

        return img_tensor, self.labels[idx]

In [5]:
train_file = 'bernie_interviews/data/train.txt'
val_file = 'bernie_interviews/data/val.txt'
test_file = 'bernie_interviews/data/test.txt'

In [6]:
def read_file(filename):
    paths = []
    labels = []
    with open(filename, 'r') as f:
        for line in f.readlines():
            video, image, label = line.split(' ')
            paths.append('bernie_interviews/images/{}/{:04d}.jpg'.format(video, int(image)))
            labels.append(int(label))
            
    return paths, labels

In [7]:
train_paths, Y_train = read_file(train_file)
val_paths, Y_val = read_file(val_file)
test_paths, Y_test = read_file(test_file)

In [8]:
def class_balance(filename):
    labels = []
    with open(filename, 'r') as f:
        for line in f.readlines():
            video, image, label = line.split(' ')
            labels.append(int(label))
    
    cb = len([lb for lb in labels if lb == 1]) / len(labels)
    
    return cb

print(class_balance(train_file))
print(class_balance(val_file))
print(class_balance(test_file))

0.03862472567666423
0.02742894910773298
0.03003087285994948


In [9]:
image_datasets = {
    'train': ImageDataset(train_paths, Y_train, transform = data_transforms['train']),
    'val_train': ImageDataset(val_paths, Y_val, transform = data_transforms['train']),
    'val': ImageDataset(val_paths, Y_val, transform = data_transforms['val']),
    'test': ImageDataset(test_paths, Y_test, transform = data_transforms['val'])
}

In [10]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                              batch_size=4,
                                              shuffle='train' in x,
                                              num_workers=4,
                                              pin_memory=True)
                                              for x in image_datasets}

In [11]:
dataset_sizes = {
    x: len(image_datasets[x])
    for x in image_datasets
}

In [12]:
dataset_sizes

{'train': 6835, 'val_train': 3026, 'val': 3026, 'test': 3563}

In [13]:
def safe_divide(a, b):
    return a / b if b > 0 else 0

In [14]:
def train_model(model, criterion, optimizer, scheduler, train_dl, val_dl, test_dl=None,
                num_epochs=25, return_best=False, verbose=True, log_file=None):
    print(train_dl, val_dl, test_dl)
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_epoch = 0
    best_test_acc = 0.0
    best_f1 = 0.0
    best_precision = 0.0
    best_recall = 0.0
    
    phases = ['train', 'val', 'test'] if test_dl is not None else ['train', 'val']

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in phases:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
                dl = dataloaders[train_dl]
                dataset_size = dataset_sizes[train_dl]
                
            elif phase == 'val':
                model.eval()   # Set model to evaluate mode
                dl = dataloaders[val_dl]
                dataset_size = dataset_sizes[val_dl]
            else:
                model.eval()
                dl = dataloaders[test_dl]
                dataset_size = dataset_sizes[test_dl]

            running_loss = 0.0
            running_corrects = 0
            true_positives = 0.
            true_negatives = 0.
            false_positives = 0.
            false_negatives = 0.

            # Iterate over data.
            for inputs, labels in dl:
                inputs = inputs.to(device)
                labels = labels.to(device).float()

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    preds = torch.where(
                        outputs >= 0.,
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                    target = torch.where(
                        labels >= 0.5,
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device)
                    )
                    loss = criterion(outputs.view(target.shape), target)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                label_vals = torch.where(
                    labels >= 0.5,
                    torch.tensor([1.]).to(device),
                    torch.tensor([0.]).to(device)
                )
                correct = preds.view(label_vals.shape) == label_vals.data
                running_corrects += torch.sum(correct)
                
                true_positives += torch.sum(
                    torch.where(
                        (correct == 1.) * (label_vals == 1.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
                true_negatives += torch.sum(
                    torch.where(
                        (correct == 1.) * (label_vals == 0.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
                false_positives += torch.sum(
                    torch.where(
                        (correct == 0.) * (label_vals == 0.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
                false_negatives += torch.sum(
                    torch.where(
                        (correct == 0.) * (label_vals == 1.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
            
            epoch_loss = running_loss / dataset_size
            epoch_acc = running_corrects.double() / dataset_size
            epoch_pre = safe_divide(true_positives, (true_positives + false_positives))
            epoch_recall = safe_divide(true_positives, (true_positives + false_negatives))
            epoch_f1 = safe_divide(2 * epoch_pre * epoch_recall, (epoch_pre + epoch_recall))

            if verbose:
                print('{} Loss: {:.4f} Acc: {:.4f} Pre: {:.4f} Rec: {:.4f} F1: {:.4f}'.format(
                    phase, epoch_loss, epoch_acc, epoch_pre, epoch_recall, epoch_f1))
                print('TP: {} TN: {} FP: {} FN: {}'.format(
                    true_positives.data, true_negatives.data, false_positives.data, false_negatives.data))
            if log_file is not None:
                log_file.write('Phase: {0}\t'
                               'Epoch: [{1}/{2}]\t'
                               'Loss: {loss_c:.4f}\t'
                               'Acc: {acc:.4f}\t'
                               'Pre: {pre:.4f}\t'
                               'Rec: {rec:.4f}\t'
                               'F1: {f1:.4f}\t'
                               'TP: {tp} '
                               'TN: {tn} '
                               'FP: {fp} '
                               'FN: {fn}\n'.format(
                                   phase, epoch + 1, num_epochs, loss_c=epoch_loss,
                                   acc=epoch_acc, pre=epoch_pre, rec=epoch_recall,
                                   f1=epoch_f1, tp=int(true_positives.data), tn=int(true_negatives.data),
                                   fp=int(false_positives.data), fn=int(false_negatives.data)
                               ))
                log_file.flush()

            # deep copy the model
            if phase == 'val' and epoch_f1 > best_f1:
                best_acc = epoch_acc
                best_f1 = epoch_f1
                best_precision = epoch_pre
                best_recall = epoch_recall
                best_epoch = epoch
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'test' and best_epoch == epoch:
                best_test_acc = epoch_acc

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    
    if return_best:
        print('Best epoch: {}'.format(best_epoch))
        print('Best val Acc: {:4f}'.format(best_acc))
        print('Best val Pre: {:4f}'.format(best_precision))
        print('Best val Rec: {:4f}'.format(best_recall))
        print('Best val F1: {:4f}'.format(best_f1))
        print('Test Acc: {:4f}'.format(best_test_acc))

        # load best model weights
        model.load_state_dict(best_model_wts)
    return model

# Traditional Supervision

In [ ]:
path = 'models/transfer_learning_tutorial_correct_train'
for seed in range(5):
    torch.manual_seed(seed)
    model_ts = models.resnet50(pretrained=True)
    num_ftrs = model_ts.fc.in_features
    model_ts.fc = nn.Linear(num_ftrs, 1)

    model_ts = model_ts.to(device)

    criterion = nn.BCEWithLogitsLoss().to(device)

    # Observe that all parameters are being optimized
    optimizer_ts = optim.SGD(model_ts.parameters(), lr=0.001, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler_ts = lr_scheduler.StepLR(optimizer_ts, step_size=7, gamma=0.1)
    
    if not os.path.exists(path):
        os.makedirs(path)
    with open(os.path.join(path, 'seed_{}.log'.format(seed)), 'w') as log_file:
        model_ts = train_model(model_ts, criterion, optimizer_ts, exp_lr_scheduler_ts,
                               'val_train', 'val', test_dl='test', num_epochs=25, verbose=True,
                               log_file=log_file, return_best=False)
        torch.save(model_ts.state_dict(), os.path.join(path, 'seed_{}.pth'.format(seed)))

```
val_train val test
Epoch 0/24
----------
/lfs/1/danfu/anaconda3/envs/sequential_ws/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:82: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
train Loss: 0.0931 Acc: 0.9736 Pre: 0.5714 Rec: 0.1446 F1: 0.2308
TP: 12.0 TN: 2934.0 FP: 9.0 FN: 71.0
val Loss: 0.1016 Acc: 0.9603 Pre: 0.4031 Rec: 0.9277 F1: 0.5620
TP: 77.0 TN: 2829.0 FP: 114.0 FN: 6.0
test Loss: 0.1077 Acc: 0.9531 Pre: 0.3780 Rec: 0.8692 F1: 0.5269
TP: 93.0 TN: 3303.0 FP: 153.0 FN: 14.0

Epoch 1/24
----------
train Loss: 0.0522 Acc: 0.9868 Pre: 0.9574 Rec: 0.5422 F1: 0.6923
TP: 45.0 TN: 2941.0 FP: 2.0 FN: 38.0
val Loss: 0.0615 Acc: 0.9795 Pre: 0.5778 Rec: 0.9398 F1: 0.7156
TP: 78.0 TN: 2886.0 FP: 57.0 FN: 5.0
test Loss: 0.0718 Acc: 0.9753 Pre: 0.5537 Rec: 0.9159 F1: 0.6901
TP: 98.0 TN: 3377.0 FP: 79.0 FN: 9.0

Epoch 2/24
----------
train Loss: 0.0384 Acc: 0.9914 Pre: 0.9831 Rec: 0.6988 F1: 0.8169
TP: 58.0 TN: 2942.0 FP: 1.0 FN: 25.0
val Loss: 0.0245 Acc: 0.9921 Pre: 0.8041 Rec: 0.9398 F1: 0.8667
TP: 78.0 TN: 2924.0 FP: 19.0 FN: 5.0
test Loss: 0.0533 Acc: 0.9823 Pre: 0.6930 Rec: 0.7383 F1: 0.7149
TP: 79.0 TN: 3421.0 FP: 35.0 FN: 28.0

Epoch 3/24
----------
train Loss: 0.0420 Acc: 0.9891 Pre: 0.8788 Rec: 0.6988 F1: 0.7785
TP: 58.0 TN: 2935.0 FP: 8.0 FN: 25.0
val Loss: 0.0112 Acc: 0.9967 Pre: 0.9506 Rec: 0.9277 F1: 0.9390
TP: 77.0 TN: 2939.0 FP: 4.0 FN: 6.0
test Loss: 0.0615 Acc: 0.9832 Pre: 0.7043 Rec: 0.7570 F1: 0.7297
TP: 81.0 TN: 3422.0 FP: 34.0 FN: 26.0

Epoch 4/24
----------
train Loss: 0.0241 Acc: 0.9944 Pre: 0.9714 Rec: 0.8193 F1: 0.8889
TP: 68.0 TN: 2941.0 FP: 2.0 FN: 15.0
val Loss: 0.0250 Acc: 0.9921 Pre: 0.7864 Rec: 0.9759 F1: 0.8710
TP: 81.0 TN: 2921.0 FP: 22.0 FN: 2.0
test Loss: 0.0630 Acc: 0.9812 Pre: 0.6235 Rec: 0.9439 F1: 0.7509
TP: 101.0 TN: 3395.0 FP: 61.0 FN: 6.0

Epoch 5/24
----------
train Loss: 0.0383 Acc: 0.9898 Pre: 0.9333 Rec: 0.6747 F1: 0.7832
TP: 56.0 TN: 2939.0 FP: 4.0 FN: 27.0
val Loss: 0.0466 Acc: 0.9845 Pre: 0.6406 Rec: 0.9880 F1: 0.7773
TP: 82.0 TN: 2897.0 FP: 46.0 FN: 1.0
test Loss: 0.0689 Acc: 0.9759 Pre: 0.5593 Rec: 0.9252 F1: 0.6972
TP: 99.0 TN: 3378.0 FP: 78.0 FN: 8.0

Epoch 6/24
----------
train Loss: 0.0255 Acc: 0.9937 Pre: 0.9706 Rec: 0.7952 F1: 0.8742
TP: 66.0 TN: 2941.0 FP: 2.0 FN: 17.0
val Loss: 0.0160 Acc: 0.9960 Pre: 0.8817 Rec: 0.9880 F1: 0.9318
TP: 82.0 TN: 2932.0 FP: 11.0 FN: 1.0
test Loss: 0.0328 Acc: 0.9913 Pre: 0.8115 Rec: 0.9252 F1: 0.8646
TP: 99.0 TN: 3433.0 FP: 23.0 FN: 8.0

Epoch 7/24
----------
train Loss: 0.0293 Acc: 0.9931 Pre: 0.9429 Rec: 0.7952 F1: 0.8627
TP: 66.0 TN: 2939.0 FP: 4.0 FN: 17.0
val Loss: 0.0180 Acc: 0.9950 Pre: 0.8469 Rec: 1.0000 F1: 0.9171
TP: 83.0 TN: 2928.0 FP: 15.0 FN: 0.0
test Loss: 0.0405 Acc: 0.9882 Pre: 0.7444 Rec: 0.9252 F1: 0.8250
TP: 99.0 TN: 3422.0 FP: 34.0 FN: 8.0

Epoch 8/24
----------
train Loss: 0.0196 Acc: 0.9950 Pre: 0.9595 Rec: 0.8554 F1: 0.9045
TP: 71.0 TN: 2940.0 FP: 3.0 FN: 12.0
val Loss: 0.0258 Acc: 0.9911 Pre: 0.7545 Rec: 1.0000 F1: 0.8601
TP: 83.0 TN: 2916.0 FP: 27.0 FN: 0.0
test Loss: 0.0545 Acc: 0.9829 Pre: 0.6513 Rec: 0.9252 F1: 0.7645
TP: 99.0 TN: 3403.0 FP: 53.0 FN: 8.0

Epoch 9/24
----------
train Loss: 0.0177 Acc: 0.9954 Pre: 1.0000 Rec: 0.8313 F1: 0.9079
TP: 69.0 TN: 2943.0 FP: 0.0 FN: 14.0
val Loss: 0.0130 Acc: 0.9957 Pre: 0.8646 Rec: 1.0000 F1: 0.9274
TP: 83.0 TN: 2930.0 FP: 13.0 FN: 0.0
test Loss: 0.0365 Acc: 0.9891 Pre: 0.7615 Rec: 0.9252 F1: 0.8354
TP: 99.0 TN: 3425.0 FP: 31.0 FN: 8.0

Epoch 10/24
----------
train Loss: 0.0162 Acc: 0.9954 Pre: 0.9600 Rec: 0.8675 F1: 0.9114
TP: 72.0 TN: 2940.0 FP: 3.0 FN: 11.0
val Loss: 0.0285 Acc: 0.9917 Pre: 0.7685 Rec: 1.0000 F1: 0.8691
TP: 83.0 TN: 2918.0 FP: 25.0 FN: 0.0
test Loss: 0.0514 Acc: 0.9848 Pre: 0.6828 Rec: 0.9252 F1: 0.7857
TP: 99.0 TN: 3410.0 FP: 46.0 FN: 8.0

Epoch 11/24
----------
train Loss: 0.0170 Acc: 0.9937 Pre: 0.9571 Rec: 0.8072 F1: 0.8758
TP: 67.0 TN: 2940.0 FP: 3.0 FN: 16.0
val Loss: 0.0212 Acc: 0.9927 Pre: 0.7905 Rec: 1.0000 F1: 0.8830
TP: 83.0 TN: 2921.0 FP: 22.0 FN: 0.0
test Loss: 0.0482 Acc: 0.9851 Pre: 0.6875 Rec: 0.9252 F1: 0.7888
TP: 99.0 TN: 3411.0 FP: 45.0 FN: 8.0

Epoch 12/24
----------
train Loss: 0.0161 Acc: 0.9950 Pre: 0.9722 Rec: 0.8434 F1: 0.9032
TP: 70.0 TN: 2941.0 FP: 2.0 FN: 13.0
val Loss: 0.0201 Acc: 0.9931 Pre: 0.7981 Rec: 1.0000 F1: 0.8877
TP: 83.0 TN: 2922.0 FP: 21.0 FN: 0.0
test Loss: 0.0454 Acc: 0.9865 Pre: 0.7122 Rec: 0.9252 F1: 0.8049
TP: 99.0 TN: 3416.0 FP: 40.0 FN: 8.0

Epoch 13/24
----------
train Loss: 0.0158 Acc: 0.9964 Pre: 0.9737 Rec: 0.8916 F1: 0.9308
TP: 74.0 TN: 2941.0 FP: 2.0 FN: 9.0
val Loss: 0.0170 Acc: 0.9950 Pre: 0.8469 Rec: 1.0000 F1: 0.9171
TP: 83.0 TN: 2928.0 FP: 15.0 FN: 0.0
test Loss: 0.0434 Acc: 0.9851 Pre: 0.6875 Rec: 0.9252 F1: 0.7888
TP: 99.0 TN: 3411.0 FP: 45.0 FN: 8.0

Epoch 14/24
----------
train Loss: 0.0173 Acc: 0.9967 Pre: 0.9867 Rec: 0.8916 F1: 0.9367
TP: 74.0 TN: 2942.0 FP: 1.0 FN: 9.0
val Loss: 0.0198 Acc: 0.9941 Pre: 0.8218 Rec: 1.0000 F1: 0.9022
TP: 83.0 TN: 2925.0 FP: 18.0 FN: 0.0
test Loss: 0.0425 Acc: 0.9862 Pre: 0.7071 Rec: 0.9252 F1: 0.8016
TP: 99.0 TN: 3415.0 FP: 41.0 FN: 8.0

Epoch 15/24
----------
train Loss: 0.0189 Acc: 0.9957 Pre: 0.9730 Rec: 0.8675 F1: 0.9172
TP: 72.0 TN: 2941.0 FP: 2.0 FN: 11.0
val Loss: 0.0245 Acc: 0.9924 Pre: 0.7830 Rec: 1.0000 F1: 0.8783
TP: 83.0 TN: 2920.0 FP: 23.0 FN: 0.0
test Loss: 0.0529 Acc: 0.9832 Pre: 0.6556 Rec: 0.9252 F1: 0.7674
TP: 99.0 TN: 3404.0 FP: 52.0 FN: 8.0

Epoch 16/24
----------
train Loss: 0.0207 Acc: 0.9950 Pre: 0.9857 Rec: 0.8313 F1: 0.9020
TP: 69.0 TN: 2942.0 FP: 1.0 FN: 14.0
val Loss: 0.0144 Acc: 0.9954 Pre: 0.8557 Rec: 1.0000 F1: 0.9222
TP: 83.0 TN: 2929.0 FP: 14.0 FN: 0.0
test Loss: 0.0373 Acc: 0.9871 Pre: 0.7226 Rec: 0.9252 F1: 0.8115
TP: 99.0 TN: 3418.0 FP: 38.0 FN: 8.0

Epoch 17/24
----------
train Loss: 0.0158 Acc: 0.9950 Pre: 0.9857 Rec: 0.8313 F1: 0.9020
TP: 69.0 TN: 2942.0 FP: 1.0 FN: 14.0
val Loss: 0.0179 Acc: 0.9950 Pre: 0.8469 Rec: 1.0000 F1: 0.9171
TP: 83.0 TN: 2928.0 FP: 15.0 FN: 0.0
test Loss: 0.0445 Acc: 0.9860 Pre: 0.7021 Rec: 0.9252 F1: 0.7984
TP: 99.0 TN: 3414.0 FP: 42.0 FN: 8.0

Epoch 18/24
----------
train Loss: 0.0145 Acc: 0.9964 Pre: 0.9865 Rec: 0.8795 F1: 0.9299
TP: 73.0 TN: 2942.0 FP: 1.0 FN: 10.0
val Loss: 0.0092 Acc: 0.9967 Pre: 0.8925 Rec: 1.0000 F1: 0.9432
TP: 83.0 TN: 2933.0 FP: 10.0 FN: 0.0
test Loss: 0.0338 Acc: 0.9896 Pre: 0.7734 Rec: 0.9252 F1: 0.8426
TP: 99.0 TN: 3427.0 FP: 29.0 FN: 8.0

Epoch 19/24
----------
train Loss: 0.0124 Acc: 0.9964 Pre: 1.0000 Rec: 0.8675 F1: 0.9290
TP: 72.0 TN: 2943.0 FP: 0.0 FN: 11.0
val Loss: 0.0219 Acc: 0.9927 Pre: 0.7905 Rec: 1.0000 F1: 0.8830
TP: 83.0 TN: 2921.0 FP: 22.0 FN: 0.0
test Loss: 0.0509 Acc: 0.9829 Pre: 0.6513 Rec: 0.9252 F1: 0.7645
TP: 99.0 TN: 3403.0 FP: 53.0 FN: 8.0

Epoch 20/24
----------
train Loss: 0.0130 Acc: 0.9974 Pre: 0.9870 Rec: 0.9157 F1: 0.9500
TP: 76.0 TN: 2942.0 FP: 1.0 FN: 7.0
val Loss: 0.0146 Acc: 0.9960 Pre: 0.8737 Rec: 1.0000 F1: 0.9326
TP: 83.0 TN: 2931.0 FP: 12.0 FN: 0.0
test Loss: 0.0377 Acc: 0.9879 Pre: 0.7388 Rec: 0.9252 F1: 0.8216
TP: 99.0 TN: 3421.0 FP: 35.0 FN: 8.0

Epoch 21/24
----------
train Loss: 0.0157 Acc: 0.9964 Pre: 0.9865 Rec: 0.8795 F1: 0.9299
TP: 73.0 TN: 2942.0 FP: 1.0 FN: 10.0
val Loss: 0.0150 Acc: 0.9964 Pre: 0.8830 Rec: 1.0000 F1: 0.9379
TP: 83.0 TN: 2932.0 FP: 11.0 FN: 0.0
test Loss: 0.0423 Acc: 0.9862 Pre: 0.7071 Rec: 0.9252 F1: 0.8016
TP: 99.0 TN: 3415.0 FP: 41.0 FN: 8.0

Epoch 22/24
----------
train Loss: 0.0135 Acc: 0.9960 Pre: 0.9863 Rec: 0.8675 F1: 0.9231
TP: 72.0 TN: 2942.0 FP: 1.0 FN: 11.0
val Loss: 0.0323 Acc: 0.9891 Pre: 0.7155 Rec: 1.0000 F1: 0.8342
TP: 83.0 TN: 2910.0 FP: 33.0 FN: 0.0
test Loss: 0.0606 Acc: 0.9804 Pre: 0.6149 Rec: 0.9252 F1: 0.7388
TP: 99.0 TN: 3394.0 FP: 62.0 FN: 8.0

Epoch 23/24
----------
train Loss: 0.0154 Acc: 0.9950 Pre: 0.9595 Rec: 0.8554 F1: 0.9045
TP: 71.0 TN: 2940.0 FP: 3.0 FN: 12.0
val Loss: 0.0255 Acc: 0.9924 Pre: 0.7830 Rec: 1.0000 F1: 0.8783
TP: 83.0 TN: 2920.0 FP: 23.0 FN: 0.0
test Loss: 0.0546 Acc: 0.9815 Pre: 0.6306 Rec: 0.9252 F1: 0.7500
TP: 99.0 TN: 3398.0 FP: 58.0 FN: 8.0

Epoch 24/24
----------
train Loss: 0.0128 Acc: 0.9954 Pre: 1.0000 Rec: 0.8313 F1: 0.9079
TP: 69.0 TN: 2943.0 FP: 0.0 FN: 14.0
val Loss: 0.0108 Acc: 0.9974 Pre: 0.9121 Rec: 1.0000 F1: 0.9540
TP: 83.0 TN: 2935.0 FP: 8.0 FN: 0.0
test Loss: 0.0335 Acc: 0.9896 Pre: 0.7734 Rec: 0.9252 F1: 0.8426
TP: 99.0 TN: 3427.0 FP: 29.0 FN: 8.0

Training complete in 28m 25s
val_train val test
Epoch 0/24
----------
train Loss: 0.0976 Acc: 0.9742 Pre: 0.7273 Rec: 0.0964 F1: 0.1702
TP: 8.0 TN: 2940.0 FP: 3.0 FN: 75.0
val Loss: 0.0412 Acc: 0.9941 Pre: 0.9114 Rec: 0.8675 F1: 0.8889
TP: 72.0 TN: 2936.0 FP: 7.0 FN: 11.0
test Loss: 0.0581 Acc: 0.9865 Pre: 0.8734 Rec: 0.6449 F1: 0.7419
TP: 69.0 TN: 3446.0 FP: 10.0 FN: 38.0

Epoch 1/24
----------
train Loss: 0.0495 Acc: 0.9868 Pre: 0.9216 Rec: 0.5663 F1: 0.7015
TP: 47.0 TN: 2939.0 FP: 4.0 FN: 36.0
val Loss: 0.0510 Acc: 0.9868 Pre: 0.6838 Rec: 0.9639 F1: 0.8000
TP: 80.0 TN: 2906.0 FP: 37.0 FN: 3.0
test Loss: 0.0747 Acc: 0.9756 Pre: 0.5581 Rec: 0.8972 F1: 0.6882
TP: 96.0 TN: 3380.0 FP: 76.0 FN: 11.0

Epoch 2/24
----------
train Loss: 0.0421 Acc: 0.9874 Pre: 0.8462 Rec: 0.6627 F1: 0.7432
TP: 55.0 TN: 2933.0 FP: 10.0 FN: 28.0
val Loss: 0.0617 Acc: 0.9782 Pre: 0.5603 Rec: 0.9518 F1: 0.7054
TP: 79.0 TN: 2881.0 FP: 62.0 FN: 4.0
test Loss: 0.0928 Acc: 0.9632 Pre: 0.4362 Rec: 0.7664 F1: 0.5559
TP: 82.0 TN: 3350.0 FP: 106.0 FN: 25.0

Epoch 3/24
----------
train Loss: 0.0440 Acc: 0.9874 Pre: 0.8814 Rec: 0.6265 F1: 0.7324
TP: 52.0 TN: 2936.0 FP: 7.0 FN: 31.0
val Loss: 0.0129 Acc: 0.9974 Pre: 0.9310 Rec: 0.9759 F1: 0.9529
TP: 81.0 TN: 2937.0 FP: 6.0 FN: 2.0
test Loss: 0.0491 Acc: 0.9806 Pre: 0.6557 Rec: 0.7477 F1: 0.6987
TP: 80.0 TN: 3414.0 FP: 42.0 FN: 27.0

Epoch 4/24
----------
train Loss: 0.0364 Acc: 0.9901 Pre: 0.9344 Rec: 0.6867 F1: 0.7917
TP: 57.0 TN: 2939.0 FP: 4.0 FN: 26.0
val Loss: 0.0186 Acc: 0.9947 Pre: 0.8454 Rec: 0.9880 F1: 0.9111
TP: 82.0 TN: 2928.0 FP: 15.0 FN: 1.0
test Loss: 0.0580 Acc: 0.9846 Pre: 0.7826 Rec: 0.6729 F1: 0.7236
TP: 72.0 TN: 3436.0 FP: 20.0 FN: 35.0

Epoch 5/24
----------
train Loss: 0.0323 Acc: 0.9911 Pre: 0.8889 Rec: 0.7711 F1: 0.8258
TP: 64.0 TN: 2935.0 FP: 8.0 FN: 19.0
val Loss: 0.0114 Acc: 0.9970 Pre: 0.9111 Rec: 0.9880 F1: 0.9480
TP: 82.0 TN: 2935.0 FP: 8.0 FN: 1.0
test Loss: 0.0487 Acc: 0.9823 Pre: 0.6746 Rec: 0.7944 F1: 0.7296
TP: 85.0 TN: 3415.0 FP: 41.0 FN: 22.0

Epoch 6/24
----------
train Loss: 0.0221 Acc: 0.9941 Pre: 0.9710 Rec: 0.8072 F1: 0.8816
TP: 67.0 TN: 2941.0 FP: 2.0 FN: 16.0
val Loss: 0.0062 Acc: 0.9980 Pre: 0.9425 Rec: 0.9880 F1: 0.9647
TP: 82.0 TN: 2938.0 FP: 5.0 FN: 1.0
test Loss: 0.0463 Acc: 0.9843 Pre: 0.7339 Rec: 0.7477 F1: 0.7407
TP: 80.0 TN: 3427.0 FP: 29.0 FN: 27.0

Epoch 7/24
----------
train Loss: 0.0135 Acc: 0.9954 Pre: 0.9600 Rec: 0.8675 F1: 0.9114
TP: 72.0 TN: 2940.0 FP: 3.0 FN: 11.0
val Loss: 0.0090 Acc: 0.9974 Pre: 0.9213 Rec: 0.9880 F1: 0.9535
TP: 82.0 TN: 2936.0 FP: 7.0 FN: 1.0
test Loss: 0.0457 Acc: 0.9832 Pre: 0.6880 Rec: 0.8037 F1: 0.7414
TP: 86.0 TN: 3417.0 FP: 39.0 FN: 21.0

Epoch 8/24
----------
train Loss: 0.0233 Acc: 0.9941 Pre: 1.0000 Rec: 0.7831 F1: 0.8784
TP: 65.0 TN: 2943.0 FP: 0.0 FN: 18.0
val Loss: 0.0083 Acc: 0.9977 Pre: 0.9318 Rec: 0.9880 F1: 0.9591
TP: 82.0 TN: 2937.0 FP: 6.0 FN: 1.0
test Loss: 0.0484 Acc: 0.9834 Pre: 0.6967 Rec: 0.7944 F1: 0.7424
TP: 85.0 TN: 3419.0 FP: 37.0 FN: 22.0

Epoch 9/24
----------
train Loss: 0.0117 Acc: 0.9970 Pre: 0.9868 Rec: 0.9036 F1: 0.9434
TP: 75.0 TN: 2942.0 FP: 1.0 FN: 8.0
val Loss: 0.0115 Acc: 0.9974 Pre: 0.9213 Rec: 0.9880 F1: 0.9535
TP: 82.0 TN: 2936.0 FP: 7.0 FN: 1.0
test Loss: 0.0485 Acc: 0.9832 Pre: 0.6794 Rec: 0.8318 F1: 0.7479
TP: 89.0 TN: 3414.0 FP: 42.0 FN: 18.0

Epoch 10/24
----------
train Loss: 0.0194 Acc: 0.9944 Pre: 0.9583 Rec: 0.8313 F1: 0.8903
TP: 69.0 TN: 2940.0 FP: 3.0 FN: 14.0
val Loss: 0.0109 Acc: 0.9970 Pre: 0.9111 Rec: 0.9880 F1: 0.9480
TP: 82.0 TN: 2935.0 FP: 8.0 FN: 1.0
test Loss: 0.0395 Acc: 0.9857 Pre: 0.7090 Rec: 0.8879 F1: 0.7884
TP: 95.0 TN: 3417.0 FP: 39.0 FN: 12.0

Epoch 11/24
----------
train Loss: 0.0218 Acc: 0.9947 Pre: 0.9855 Rec: 0.8193 F1: 0.8947
TP: 68.0 TN: 2942.0 FP: 1.0 FN: 15.0
val Loss: 0.0080 Acc: 0.9974 Pre: 0.9213 Rec: 0.9880 F1: 0.9535
TP: 82.0 TN: 2936.0 FP: 7.0 FN: 1.0
test Loss: 0.0371 Acc: 0.9860 Pre: 0.7317 Rec: 0.8411 F1: 0.7826
TP: 90.0 TN: 3423.0 FP: 33.0 FN: 17.0

Epoch 12/24
----------
train Loss: 0.0247 Acc: 0.9937 Pre: 0.9571 Rec: 0.8072 F1: 0.8758
TP: 67.0 TN: 2940.0 FP: 3.0 FN: 16.0
val Loss: 0.0101 Acc: 0.9970 Pre: 0.9111 Rec: 0.9880 F1: 0.9480
TP: 82.0 TN: 2935.0 FP: 8.0 FN: 1.0
test Loss: 0.0403 Acc: 0.9851 Pre: 0.7143 Rec: 0.8411 F1: 0.7725
TP: 90.0 TN: 3420.0 FP: 36.0 FN: 17.0

Epoch 13/24
----------
train Loss: 0.0175 Acc: 0.9941 Pre: 0.9114 Rec: 0.8675 F1: 0.8889
TP: 72.0 TN: 2936.0 FP: 7.0 FN: 11.0
val Loss: 0.0056 Acc: 0.9990 Pre: 0.9762 Rec: 0.9880 F1: 0.9820
TP: 82.0 TN: 2941.0 FP: 2.0 FN: 1.0
test Loss: 0.0354 Acc: 0.9868 Pre: 0.7727 Rec: 0.7944 F1: 0.7834
TP: 85.0 TN: 3431.0 FP: 25.0 FN: 22.0

Epoch 14/24
----------
train Loss: 0.0192 Acc: 0.9947 Pre: 0.9589 Rec: 0.8434 F1: 0.8974
TP: 70.0 TN: 2940.0 FP: 3.0 FN: 13.0
val Loss: 0.0082 Acc: 0.9974 Pre: 0.9213 Rec: 0.9880 F1: 0.9535
TP: 82.0 TN: 2936.0 FP: 7.0 FN: 1.0
test Loss: 0.0407 Acc: 0.9854 Pre: 0.7391 Rec: 0.7944 F1: 0.7658
TP: 85.0 TN: 3426.0 FP: 30.0 FN: 22.0

Epoch 15/24
----------
train Loss: 0.0181 Acc: 0.9950 Pre: 0.9595 Rec: 0.8554 F1: 0.9045
TP: 71.0 TN: 2940.0 FP: 3.0 FN: 12.0
val Loss: 0.0080 Acc: 0.9974 Pre: 0.9213 Rec: 0.9880 F1: 0.9535
TP: 82.0 TN: 2936.0 FP: 7.0 FN: 1.0
test Loss: 0.0383 Acc: 0.9865 Pre: 0.7438 Rec: 0.8411 F1: 0.7895
TP: 90.0 TN: 3425.0 FP: 31.0 FN: 17.0

Epoch 16/24
----------
train Loss: 0.0158 Acc: 0.9957 Pre: 0.9605 Rec: 0.8795 F1: 0.9182
TP: 73.0 TN: 2940.0 FP: 3.0 FN: 10.0
val Loss: 0.0089 Acc: 0.9977 Pre: 0.9318 Rec: 0.9880 F1: 0.9591
TP: 82.0 TN: 2937.0 FP: 6.0 FN: 1.0
test Loss: 0.0395 Acc: 0.9862 Pre: 0.7302 Rec: 0.8598 F1: 0.7897
TP: 92.0 TN: 3422.0 FP: 34.0 FN: 15.0

Epoch 17/24
----------
train Loss: 0.0175 Acc: 0.9947 Pre: 0.9589 Rec: 0.8434 F1: 0.8974
TP: 70.0 TN: 2940.0 FP: 3.0 FN: 13.0
val Loss: 0.0099 Acc: 0.9964 Pre: 0.8913 Rec: 0.9880 F1: 0.9371
TP: 82.0 TN: 2933.0 FP: 10.0 FN: 1.0
test Loss: 0.0379 Acc: 0.9857 Pre: 0.7222 Rec: 0.8505 F1: 0.7811
TP: 91.0 TN: 3421.0 FP: 35.0 FN: 16.0

Epoch 18/24
----------
train Loss: 0.0121 Acc: 0.9967 Pre: 0.9740 Rec: 0.9036 F1: 0.9375
TP: 75.0 TN: 2941.0 FP: 2.0 FN: 8.0
val Loss: 0.0088 Acc: 0.9977 Pre: 0.9318 Rec: 0.9880 F1: 0.9591
TP: 82.0 TN: 2937.0 FP: 6.0 FN: 1.0
test Loss: 0.0364 Acc: 0.9865 Pre: 0.7398 Rec: 0.8505 F1: 0.7913
TP: 91.0 TN: 3424.0 FP: 32.0 FN: 16.0

Epoch 19/24
----------
train Loss: 0.0162 Acc: 0.9960 Pre: 0.9863 Rec: 0.8675 F1: 0.9231
TP: 72.0 TN: 2942.0 FP: 1.0 FN: 11.0
val Loss: 0.0069 Acc: 0.9980 Pre: 0.9425 Rec: 0.9880 F1: 0.9647
TP: 82.0 TN: 2938.0 FP: 5.0 FN: 1.0
test Loss: 0.0408 Acc: 0.9848 Pre: 0.7345 Rec: 0.7757 F1: 0.7545
TP: 83.0 TN: 3426.0 FP: 30.0 FN: 24.0

Epoch 20/24
----------
train Loss: 0.0158 Acc: 0.9950 Pre: 0.9722 Rec: 0.8434 F1: 0.9032
TP: 70.0 TN: 2941.0 FP: 2.0 FN: 13.0
val Loss: 0.0059 Acc: 0.9980 Pre: 0.9425 Rec: 0.9880 F1: 0.9647
TP: 82.0 TN: 2938.0 FP: 5.0 FN: 1.0
test Loss: 0.0378 Acc: 0.9865 Pre: 0.7706 Rec: 0.7850 F1: 0.7778
TP: 84.0 TN: 3431.0 FP: 25.0 FN: 23.0

Epoch 21/24
----------
train Loss: 0.0129 Acc: 0.9957 Pre: 0.9730 Rec: 0.8675 F1: 0.9172
TP: 72.0 TN: 2941.0 FP: 2.0 FN: 11.0
val Loss: 0.0125 Acc: 0.9957 Pre: 0.8723 Rec: 0.9880 F1: 0.9266
TP: 82.0 TN: 2931.0 FP: 12.0 FN: 1.0
test Loss: 0.0424 Acc: 0.9851 Pre: 0.7015 Rec: 0.8785 F1: 0.7801
TP: 94.0 TN: 3416.0 FP: 40.0 FN: 13.0

Epoch 22/24
----------
train Loss: 0.0147 Acc: 0.9944 Pre: 0.9714 Rec: 0.8193 F1: 0.8889
TP: 68.0 TN: 2941.0 FP: 2.0 FN: 15.0
val Loss: 0.0069 Acc: 0.9980 Pre: 0.9425 Rec: 0.9880 F1: 0.9647
TP: 82.0 TN: 2938.0 FP: 5.0 FN: 1.0
test Loss: 0.0360 Acc: 0.9868 Pre: 0.7500 Rec: 0.8411 F1: 0.7930
TP: 90.0 TN: 3426.0 FP: 30.0 FN: 17.0

Epoch 23/24
----------
train Loss: 0.0162 Acc: 0.9957 Pre: 0.9605 Rec: 0.8795 F1: 0.9182
TP: 73.0 TN: 2940.0 FP: 3.0 FN: 10.0
val Loss: 0.0138 Acc: 0.9954 Pre: 0.8632 Rec: 0.9880 F1: 0.9213
TP: 82.0 TN: 2930.0 FP: 13.0 FN: 1.0
test Loss: 0.0424 Acc: 0.9865 Pre: 0.7185 Rec: 0.9065 F1: 0.8017
TP: 97.0 TN: 3418.0 FP: 38.0 FN: 10.0

Epoch 24/24
----------
train Loss: 0.0178 Acc: 0.9950 Pre: 0.9722 Rec: 0.8434 F1: 0.9032
TP: 70.0 TN: 2941.0 FP: 2.0 FN: 13.0
val Loss: 0.0186 Acc: 0.9931 Pre: 0.8039 Rec: 0.9880 F1: 0.8865
TP: 82.0 TN: 2923.0 FP: 20.0 FN: 1.0
test Loss: 0.0471 Acc: 0.9829 Pre: 0.6667 Rec: 0.8598 F1: 0.7510
TP: 92.0 TN: 3410.0 FP: 46.0 FN: 15.0

Training complete in 28m 23s
val_train val test
Epoch 0/24
----------
train Loss: 0.1025 Acc: 0.9726 Pre: 0.5000 Rec: 0.0120 F1: 0.0235
TP: 1.0 TN: 2942.0 FP: 1.0 FN: 82.0
val Loss: 0.1230 Acc: 0.9610 Pre: 0.4054 Rec: 0.9036 F1: 0.5597
TP: 75.0 TN: 2833.0 FP: 110.0 FN: 8.0
test Loss: 0.1252 Acc: 0.9613 Pre: 0.4317 Rec: 0.9159 F1: 0.5868
TP: 98.0 TN: 3327.0 FP: 129.0 FN: 9.0

Epoch 1/24
----------
train Loss: 0.0556 Acc: 0.9861 Pre: 0.9020 Rec: 0.5542 F1: 0.6866
TP: 46.0 TN: 2938.0 FP: 5.0 FN: 37.0
val Loss: 0.0198 Acc: 0.9937 Pre: 0.8636 Rec: 0.9157 F1: 0.8889
TP: 76.0 TN: 2931.0 FP: 12.0 FN: 7.0
test Loss: 0.0429 Acc: 0.9829 Pre: 0.6983 Rec: 0.7570 F1: 0.7265
TP: 81.0 TN: 3421.0 FP: 35.0 FN: 26.0

Epoch 2/24
----------
train Loss: 0.0494 Acc: 0.9865 Pre: 0.8750 Rec: 0.5904 F1: 0.7050
TP: 49.0 TN: 2936.0 FP: 7.0 FN: 34.0
val Loss: 0.0231 Acc: 0.9914 Pre: 0.7767 Rec: 0.9639 F1: 0.8602
TP: 80.0 TN: 2920.0 FP: 23.0 FN: 3.0
test Loss: 0.0410 Acc: 0.9865 Pre: 0.7252 Rec: 0.8879 F1: 0.7983
TP: 95.0 TN: 3420.0 FP: 36.0 FN: 12.0

Epoch 3/24
----------
train Loss: 0.0422 Acc: 0.9904 Pre: 0.9355 Rec: 0.6988 F1: 0.8000
TP: 58.0 TN: 2939.0 FP: 4.0 FN: 25.0
val Loss: 0.0289 Acc: 0.9924 Pre: 0.7941 Rec: 0.9759 F1: 0.8757
TP: 81.0 TN: 2922.0 FP: 21.0 FN: 2.0
test Loss: 0.0386 Acc: 0.9868 Pre: 0.7308 Rec: 0.8879 F1: 0.8017
TP: 95.0 TN: 3421.0 FP: 35.0 FN: 12.0

Epoch 4/24
----------
train Loss: 0.0316 Acc: 0.9901 Pre: 0.9077 Rec: 0.7108 F1: 0.7973
TP: 59.0 TN: 2937.0 FP: 6.0 FN: 24.0
val Loss: 0.0276 Acc: 0.9911 Pre: 0.7593 Rec: 0.9880 F1: 0.8586
TP: 82.0 TN: 2917.0 FP: 26.0 FN: 1.0
test Loss: 0.0497 Acc: 0.9843 Pre: 0.6735 Rec: 0.9252 F1: 0.7795
TP: 99.0 TN: 3408.0 FP: 48.0 FN: 8.0

Epoch 5/24
----------
train Loss: 0.0336 Acc: 0.9904 Pre: 0.9091 Rec: 0.7229 F1: 0.8054
TP: 60.0 TN: 2937.0 FP: 6.0 FN: 23.0
val Loss: 0.0167 Acc: 0.9950 Pre: 0.8469 Rec: 1.0000 F1: 0.9171
TP: 83.0 TN: 2928.0 FP: 15.0 FN: 0.0
test Loss: 0.0450 Acc: 0.9826 Pre: 0.6619 Rec: 0.8598 F1: 0.7480
TP: 92.0 TN: 3409.0 FP: 47.0 FN: 15.0

Epoch 6/24
----------
train Loss: 0.0183 Acc: 0.9954 Pre: 0.9726 Rec: 0.8554 F1: 0.9103
TP: 71.0 TN: 2941.0 FP: 2.0 FN: 12.0
val Loss: 0.0178 Acc: 0.9947 Pre: 0.8384 Rec: 1.0000 F1: 0.9121
TP: 83.0 TN: 2927.0 FP: 16.0 FN: 0.0
test Loss: 0.0459 Acc: 0.9840 Pre: 0.6812 Rec: 0.8785 F1: 0.7673
TP: 94.0 TN: 3412.0 FP: 44.0 FN: 13.0

Epoch 7/24
----------
train Loss: 0.0183 Acc: 0.9944 Pre: 1.0000 Rec: 0.7952 F1: 0.8859
TP: 66.0 TN: 2943.0 FP: 0.0 FN: 17.0
val Loss: 0.0163 Acc: 0.9944 Pre: 0.8300 Rec: 1.0000 F1: 0.9071
TP: 83.0 TN: 2926.0 FP: 17.0 FN: 0.0
test Loss: 0.0447 Acc: 0.9846 Pre: 0.6831 Rec: 0.9065 F1: 0.7791
TP: 97.0 TN: 3411.0 FP: 45.0 FN: 10.0

Epoch 8/24
----------
train Loss: 0.0260 Acc: 0.9924 Pre: 0.9545 Rec: 0.7590 F1: 0.8456
TP: 63.0 TN: 2940.0 FP: 3.0 FN: 20.0
val Loss: 0.0252 Acc: 0.9924 Pre: 0.7830 Rec: 1.0000 F1: 0.8783
TP: 83.0 TN: 2920.0 FP: 23.0 FN: 0.0
test Loss: 0.0594 Acc: 0.9775 Pre: 0.5789 Rec: 0.9252 F1: 0.7122
TP: 99.0 TN: 3384.0 FP: 72.0 FN: 8.0

Epoch 9/24
----------
train Loss: 0.0177 Acc: 0.9954 Pre: 0.9726 Rec: 0.8554 F1: 0.9103
TP: 71.0 TN: 2941.0 FP: 2.0 FN: 12.0
val Loss: 0.0252 Acc: 0.9904 Pre: 0.7411 Rec: 1.0000 F1: 0.8513
TP: 83.0 TN: 2914.0 FP: 29.0 FN: 0.0
test Loss: 0.0577 Acc: 0.9787 Pre: 0.5939 Rec: 0.9159 F1: 0.7206
TP: 98.0 TN: 3389.0 FP: 67.0 FN: 9.0

Epoch 10/24
----------
train Loss: 0.0166 Acc: 0.9967 Pre: 0.9867 Rec: 0.8916 F1: 0.9367
TP: 74.0 TN: 2942.0 FP: 1.0 FN: 9.0
val Loss: 0.0199 Acc: 0.9937 Pre: 0.8137 Rec: 1.0000 F1: 0.8973
TP: 83.0 TN: 2924.0 FP: 19.0 FN: 0.0
test Loss: 0.0486 Acc: 0.9826 Pre: 0.6510 Rec: 0.9065 F1: 0.7578
TP: 97.0 TN: 3404.0 FP: 52.0 FN: 10.0

Epoch 11/24
----------
train Loss: 0.0189 Acc: 0.9950 Pre: 0.9722 Rec: 0.8434 F1: 0.9032
TP: 70.0 TN: 2941.0 FP: 2.0 FN: 13.0
val Loss: 0.0160 Acc: 0.9950 Pre: 0.8469 Rec: 1.0000 F1: 0.9171
TP: 83.0 TN: 2928.0 FP: 15.0 FN: 0.0
test Loss: 0.0475 Acc: 0.9834 Pre: 0.6644 Rec: 0.9065 F1: 0.7668
TP: 97.0 TN: 3407.0 FP: 49.0 FN: 10.0

Epoch 12/24
----------
train Loss: 0.0242 Acc: 0.9934 Pre: 0.9315 Rec: 0.8193 F1: 0.8718
TP: 68.0 TN: 2938.0 FP: 5.0 FN: 15.0
val Loss: 0.0136 Acc: 0.9974 Pre: 0.9121 Rec: 1.0000 F1: 0.9540
TP: 83.0 TN: 2935.0 FP: 8.0 FN: 0.0
test Loss: 0.0427 Acc: 0.9848 Pre: 0.6879 Rec: 0.9065 F1: 0.7823
TP: 97.0 TN: 3412.0 FP: 44.0 FN: 10.0

Epoch 13/24
----------
train Loss: 0.0135 Acc: 0.9967 Pre: 0.9740 Rec: 0.9036 F1: 0.9375
TP: 75.0 TN: 2941.0 FP: 2.0 FN: 8.0
val Loss: 0.0092 Acc: 0.9977 Pre: 0.9222 Rec: 1.0000 F1: 0.9595
TP: 83.0 TN: 2936.0 FP: 7.0 FN: 0.0
test Loss: 0.0360 Acc: 0.9871 Pre: 0.7480 Rec: 0.8598 F1: 0.8000
TP: 92.0 TN: 3425.0 FP: 31.0 FN: 15.0

Epoch 14/24
----------
train Loss: 0.0167 Acc: 0.9957 Pre: 0.9730 Rec: 0.8675 F1: 0.9172
TP: 72.0 TN: 2941.0 FP: 2.0 FN: 11.0
val Loss: 0.0101 Acc: 0.9974 Pre: 0.9121 Rec: 1.0000 F1: 0.9540
TP: 83.0 TN: 2935.0 FP: 8.0 FN: 0.0
test Loss: 0.0370 Acc: 0.9860 Pre: 0.7111 Rec: 0.8972 F1: 0.7934
TP: 96.0 TN: 3417.0 FP: 39.0 FN: 11.0

Epoch 15/24
----------
train Loss: 0.0167 Acc: 0.9941 Pre: 0.9851 Rec: 0.7952 F1: 0.8800
TP: 66.0 TN: 2942.0 FP: 1.0 FN: 17.0
val Loss: 0.0118 Acc: 0.9977 Pre: 0.9222 Rec: 1.0000 F1: 0.9595
TP: 83.0 TN: 2936.0 FP: 7.0 FN: 0.0
test Loss: 0.0409 Acc: 0.9860 Pre: 0.7080 Rec: 0.9065 F1: 0.7951
TP: 97.0 TN: 3416.0 FP: 40.0 FN: 10.0

Epoch 16/24
----------
train Loss: 0.0180 Acc: 0.9957 Pre: 1.0000 Rec: 0.8434 F1: 0.9150
TP: 70.0 TN: 2943.0 FP: 0.0 FN: 13.0
val Loss: 0.0173 Acc: 0.9960 Pre: 0.8737 Rec: 1.0000 F1: 0.9326
TP: 83.0 TN: 2931.0 FP: 12.0 FN: 0.0
test Loss: 0.0471 Acc: 0.9832 Pre: 0.6599 Rec: 0.9065 F1: 0.7638
TP: 97.0 TN: 3406.0 FP: 50.0 FN: 10.0

Epoch 17/24
----------
train Loss: 0.0173 Acc: 0.9957 Pre: 0.9605 Rec: 0.8795 F1: 0.9182
TP: 73.0 TN: 2940.0 FP: 3.0 FN: 10.0
val Loss: 0.0088 Acc: 0.9974 Pre: 0.9121 Rec: 1.0000 F1: 0.9540
TP: 83.0 TN: 2935.0 FP: 8.0 FN: 0.0
test Loss: 0.0357 Acc: 0.9877 Pre: 0.7561 Rec: 0.8692 F1: 0.8087
TP: 93.0 TN: 3426.0 FP: 30.0 FN: 14.0

Epoch 18/24
----------
train Loss: 0.0206 Acc: 0.9947 Pre: 0.9718 Rec: 0.8313 F1: 0.8961
TP: 69.0 TN: 2941.0 FP: 2.0 FN: 14.0
val Loss: 0.0136 Acc: 0.9967 Pre: 0.8925 Rec: 1.0000 F1: 0.9432
TP: 83.0 TN: 2933.0 FP: 10.0 FN: 0.0
test Loss: 0.0445 Acc: 0.9846 Pre: 0.6806 Rec: 0.9159 F1: 0.7809
TP: 98.0 TN: 3410.0 FP: 46.0 FN: 9.0

Epoch 19/24
----------
train Loss: 0.0125 Acc: 0.9967 Pre: 0.9740 Rec: 0.9036 F1: 0.9375
TP: 75.0 TN: 2941.0 FP: 2.0 FN: 8.0
val Loss: 0.0168 Acc: 0.9954 Pre: 0.8557 Rec: 1.0000 F1: 0.9222
TP: 83.0 TN: 2929.0 FP: 14.0 FN: 0.0
test Loss: 0.0478 Acc: 0.9834 Pre: 0.6667 Rec: 0.8972 F1: 0.7649
TP: 96.0 TN: 3408.0 FP: 48.0 FN: 11.0

Epoch 20/24
----------
train Loss: 0.0158 Acc: 0.9964 Pre: 0.9500 Rec: 0.9157 F1: 0.9325
TP: 76.0 TN: 2939.0 FP: 4.0 FN: 7.0
val Loss: 0.0081 Acc: 0.9983 Pre: 0.9432 Rec: 1.0000 F1: 0.9708
TP: 83.0 TN: 2938.0 FP: 5.0 FN: 0.0
test Loss: 0.0364 Acc: 0.9882 Pre: 0.7876 Rec: 0.8318 F1: 0.8091
TP: 89.0 TN: 3432.0 FP: 24.0 FN: 18.0

Epoch 21/24
----------
train Loss: 0.0146 Acc: 0.9954 Pre: 0.9600 Rec: 0.8675 F1: 0.9114
TP: 72.0 TN: 2940.0 FP: 3.0 FN: 11.0
val Loss: 0.0124 Acc: 0.9967 Pre: 0.8925 Rec: 1.0000 F1: 0.9432
TP: 83.0 TN: 2933.0 FP: 10.0 FN: 0.0
test Loss: 0.0422 Acc: 0.9857 Pre: 0.7059 Rec: 0.8972 F1: 0.7901
TP: 96.0 TN: 3416.0 FP: 40.0 FN: 11.0

Epoch 22/24
----------
train Loss: 0.0145 Acc: 0.9967 Pre: 1.0000 Rec: 0.8795 F1: 0.9359
TP: 73.0 TN: 2943.0 FP: 0.0 FN: 10.0
val Loss: 0.0098 Acc: 0.9980 Pre: 0.9326 Rec: 1.0000 F1: 0.9651
TP: 83.0 TN: 2937.0 FP: 6.0 FN: 0.0
test Loss: 0.0379 Acc: 0.9862 Pre: 0.7197 Rec: 0.8879 F1: 0.7950
TP: 95.0 TN: 3419.0 FP: 37.0 FN: 12.0

Epoch 23/24
----------
train Loss: 0.0177 Acc: 0.9954 Pre: 1.0000 Rec: 0.8313 F1: 0.9079
TP: 69.0 TN: 2943.0 FP: 0.0 FN: 14.0
val Loss: 0.0167 Acc: 0.9957 Pre: 0.8646 Rec: 1.0000 F1: 0.9274
TP: 83.0 TN: 2930.0 FP: 13.0 FN: 0.0
test Loss: 0.0495 Acc: 0.9823 Pre: 0.6528 Rec: 0.8785 F1: 0.7490
TP: 94.0 TN: 3406.0 FP: 50.0 FN: 13.0

Epoch 24/24
----------
train Loss: 0.0141 Acc: 0.9957 Pre: 0.9730 Rec: 0.8675 F1: 0.9172
TP: 72.0 TN: 2941.0 FP: 2.0 FN: 11.0
val Loss: 0.0105 Acc: 0.9967 Pre: 0.8925 Rec: 1.0000 F1: 0.9432
TP: 83.0 TN: 2933.0 FP: 10.0 FN: 0.0
test Loss: 0.0382 Acc: 0.9862 Pre: 0.7266 Rec: 0.8692 F1: 0.7915
TP: 93.0 TN: 3421.0 FP: 35.0 FN: 14.0

Training complete in 28m 57s
val_train val test
Epoch 0/24
----------
train Loss: 0.1018 Acc: 0.9749 Pre: 1.0000 Rec: 0.0843 F1: 0.1556
TP: 7.0 TN: 2943.0 FP: 0.0 FN: 76.0
val Loss: 0.0346 Acc: 0.9924 Pre: 0.9839 Rec: 0.7349 F1: 0.8414
TP: 61.0 TN: 2942.0 FP: 1.0 FN: 22.0
test Loss: 0.0624 Acc: 0.9843 Pre: 0.9636 Rec: 0.4953 F1: 0.6543
TP: 53.0 TN: 3454.0 FP: 2.0 FN: 54.0

Epoch 1/24
----------
train Loss: 0.0505 Acc: 0.9884 Pre: 0.9286 Rec: 0.6265 F1: 0.7482
TP: 52.0 TN: 2939.0 FP: 4.0 FN: 31.0
val Loss: 0.0327 Acc: 0.9901 Pre: 0.7624 Rec: 0.9277 F1: 0.8370
TP: 77.0 TN: 2919.0 FP: 24.0 FN: 6.0
test Loss: 0.0413 Acc: 0.9874 Pre: 0.7460 Rec: 0.8785 F1: 0.8069
TP: 94.0 TN: 3424.0 FP: 32.0 FN: 13.0

Epoch 2/24
----------
train Loss: 0.0412 Acc: 0.9891 Pre: 0.9032 Rec: 0.6747 F1: 0.7724
TP: 56.0 TN: 2937.0 FP: 6.0 FN: 27.0
val Loss: 0.0347 Acc: 0.9868 Pre: 0.6903 Rec: 0.9398 F1: 0.7959
TP: 78.0 TN: 2908.0 FP: 35.0 FN: 5.0
test Loss: 0.0605 Acc: 0.9770 Pre: 0.5828 Rec: 0.8224 F1: 0.6822
TP: 88.0 TN: 3393.0 FP: 63.0 FN: 19.0

Epoch 3/24
----------
train Loss: 0.0388 Acc: 0.9898 Pre: 0.9062 Rec: 0.6988 F1: 0.7891
TP: 58.0 TN: 2937.0 FP: 6.0 FN: 25.0
val Loss: 0.0115 Acc: 0.9980 Pre: 0.9639 Rec: 0.9639 F1: 0.9639
TP: 80.0 TN: 2940.0 FP: 3.0 FN: 3.0
test Loss: 0.0476 Acc: 0.9905 Pre: 0.9195 Rec: 0.7477 F1: 0.8247
TP: 80.0 TN: 3449.0 FP: 7.0 FN: 27.0

Epoch 4/24
----------
train Loss: 0.0364 Acc: 0.9904 Pre: 0.9219 Rec: 0.7108 F1: 0.8027
TP: 59.0 TN: 2938.0 FP: 5.0 FN: 24.0
val Loss: 0.0192 Acc: 0.9934 Pre: 0.8247 Rec: 0.9639 F1: 0.8889
TP: 80.0 TN: 2926.0 FP: 17.0 FN: 3.0
test Loss: 0.0409 Acc: 0.9871 Pre: 0.7293 Rec: 0.9065 F1: 0.8083
TP: 97.0 TN: 3420.0 FP: 36.0 FN: 10.0

Epoch 5/24
----------
train Loss: 0.0322 Acc: 0.9917 Pre: 0.9265 Rec: 0.7590 F1: 0.8344
TP: 63.0 TN: 2938.0 FP: 5.0 FN: 20.0
val Loss: 0.0214 Acc: 0.9954 Pre: 0.8710 Rec: 0.9759 F1: 0.9205
TP: 81.0 TN: 2931.0 FP: 12.0 FN: 2.0
test Loss: 0.0532 Acc: 0.9857 Pre: 0.7000 Rec: 0.9159 F1: 0.7935
TP: 98.0 TN: 3414.0 FP: 42.0 FN: 9.0

Epoch 6/24
----------
train Loss: 0.0284 Acc: 0.9921 Pre: 0.9836 Rec: 0.7229 F1: 0.8333
TP: 60.0 TN: 2942.0 FP: 1.0 FN: 23.0
val Loss: 0.0158 Acc: 0.9954 Pre: 0.8710 Rec: 0.9759 F1: 0.9205
TP: 81.0 TN: 2931.0 FP: 12.0 FN: 2.0
test Loss: 0.0477 Acc: 0.9860 Pre: 0.7143 Rec: 0.8879 F1: 0.7917
TP: 95.0 TN: 3418.0 FP: 38.0 FN: 12.0

Epoch 7/24
----------
train Loss: 0.0298 Acc: 0.9911 Pre: 0.9667 Rec: 0.6988 F1: 0.8112
TP: 58.0 TN: 2941.0 FP: 2.0 FN: 25.0
val Loss: 0.0194 Acc: 0.9954 Pre: 0.8632 Rec: 0.9880 F1: 0.9213
TP: 82.0 TN: 2930.0 FP: 13.0 FN: 1.0
test Loss: 0.0523 Acc: 0.9862 Pre: 0.7101 Rec: 0.9159 F1: 0.8000
TP: 98.0 TN: 3416.0 FP: 40.0 FN: 9.0

Epoch 8/24
----------
train Loss: 0.0158 Acc: 0.9970 Pre: 0.9868 Rec: 0.9036 F1: 0.9434
TP: 75.0 TN: 2942.0 FP: 1.0 FN: 8.0
val Loss: 0.0178 Acc: 0.9967 Pre: 0.9011 Rec: 0.9880 F1: 0.9425
TP: 82.0 TN: 2934.0 FP: 9.0 FN: 1.0
test Loss: 0.0549 Acc: 0.9843 Pre: 0.6759 Rec: 0.9159 F1: 0.7778
TP: 98.0 TN: 3409.0 FP: 47.0 FN: 9.0

Epoch 9/24
----------
train Loss: 0.0213 Acc: 0.9931 Pre: 0.9429 Rec: 0.7952 F1: 0.8627
TP: 66.0 TN: 2939.0 FP: 4.0 FN: 17.0
val Loss: 0.0136 Acc: 0.9964 Pre: 0.8913 Rec: 0.9880 F1: 0.9371
TP: 82.0 TN: 2933.0 FP: 10.0 FN: 1.0
test Loss: 0.0461 Acc: 0.9865 Pre: 0.7185 Rec: 0.9065 F1: 0.8017
TP: 97.0 TN: 3418.0 FP: 38.0 FN: 10.0

Epoch 10/24
----------
train Loss: 0.0197 Acc: 0.9941 Pre: 0.9577 Rec: 0.8193 F1: 0.8831
TP: 68.0 TN: 2940.0 FP: 3.0 FN: 15.0
val Loss: 0.0214 Acc: 0.9947 Pre: 0.8454 Rec: 0.9880 F1: 0.9111
TP: 82.0 TN: 2928.0 FP: 15.0 FN: 1.0
test Loss: 0.0543 Acc: 0.9846 Pre: 0.6781 Rec: 0.9252 F1: 0.7826
TP: 99.0 TN: 3409.0 FP: 47.0 FN: 8.0

Epoch 11/24
----------
train Loss: 0.0169 Acc: 0.9957 Pre: 0.9861 Rec: 0.8554 F1: 0.9161
TP: 71.0 TN: 2942.0 FP: 1.0 FN: 12.0
val Loss: 0.0164 Acc: 0.9947 Pre: 0.8454 Rec: 0.9880 F1: 0.9111
TP: 82.0 TN: 2928.0 FP: 15.0 FN: 1.0
test Loss: 0.0488 Acc: 0.9862 Pre: 0.7071 Rec: 0.9252 F1: 0.8016
TP: 99.0 TN: 3415.0 FP: 41.0 FN: 8.0

Epoch 12/24
----------
train Loss: 0.0166 Acc: 0.9941 Pre: 0.9577 Rec: 0.8193 F1: 0.8831
TP: 68.0 TN: 2940.0 FP: 3.0 FN: 15.0
val Loss: 0.0118 Acc: 0.9964 Pre: 0.8913 Rec: 0.9880 F1: 0.9371
TP: 82.0 TN: 2933.0 FP: 10.0 FN: 1.0
test Loss: 0.0434 Acc: 0.9871 Pre: 0.7259 Rec: 0.9159 F1: 0.8099
TP: 98.0 TN: 3419.0 FP: 37.0 FN: 9.0

Epoch 13/24
----------
train Loss: 0.0167 Acc: 0.9954 Pre: 0.9600 Rec: 0.8675 F1: 0.9114
TP: 72.0 TN: 2940.0 FP: 3.0 FN: 11.0
val Loss: 0.0209 Acc: 0.9947 Pre: 0.8454 Rec: 0.9880 F1: 0.9111
TP: 82.0 TN: 2928.0 FP: 15.0 FN: 1.0
test Loss: 0.0548 Acc: 0.9837 Pre: 0.6644 Rec: 0.9252 F1: 0.7734
TP: 99.0 TN: 3406.0 FP: 50.0 FN: 8.0

Epoch 14/24
----------
train Loss: 0.0179 Acc: 0.9960 Pre: 0.9733 Rec: 0.8795 F1: 0.9241
TP: 73.0 TN: 2941.0 FP: 2.0 FN: 10.0
val Loss: 0.0055 Acc: 0.9983 Pre: 0.9535 Rec: 0.9880 F1: 0.9704
TP: 82.0 TN: 2939.0 FP: 4.0 FN: 1.0
test Loss: 0.0371 Acc: 0.9902 Pre: 0.8396 Rec: 0.8318 F1: 0.8357
TP: 89.0 TN: 3439.0 FP: 17.0 FN: 18.0

Epoch 15/24
----------
train Loss: 0.0136 Acc: 0.9960 Pre: 0.9733 Rec: 0.8795 F1: 0.9241
TP: 73.0 TN: 2941.0 FP: 2.0 FN: 10.0
val Loss: 0.0174 Acc: 0.9950 Pre: 0.8542 Rec: 0.9880 F1: 0.9162
TP: 82.0 TN: 2929.0 FP: 14.0 FN: 1.0
test Loss: 0.0503 Acc: 0.9857 Pre: 0.6972 Rec: 0.9252 F1: 0.7952
TP: 99.0 TN: 3413.0 FP: 43.0 FN: 8.0

Epoch 16/24
----------
train Loss: 0.0155 Acc: 0.9950 Pre: 0.9595 Rec: 0.8554 F1: 0.9045
TP: 71.0 TN: 2940.0 FP: 3.0 FN: 12.0
val Loss: 0.0205 Acc: 0.9944 Pre: 0.8300 Rec: 1.0000 F1: 0.9071
TP: 83.0 TN: 2926.0 FP: 17.0 FN: 0.0
test Loss: 0.0569 Acc: 0.9843 Pre: 0.6735 Rec: 0.9252 F1: 0.7795
TP: 99.0 TN: 3408.0 FP: 48.0 FN: 8.0

Epoch 17/24
----------
train Loss: 0.0208 Acc: 0.9941 Pre: 0.9333 Rec: 0.8434 F1: 0.8861
TP: 70.0 TN: 2938.0 FP: 5.0 FN: 13.0
val Loss: 0.0134 Acc: 0.9964 Pre: 0.8913 Rec: 0.9880 F1: 0.9371
TP: 82.0 TN: 2933.0 FP: 10.0 FN: 1.0
test Loss: 0.0477 Acc: 0.9857 Pre: 0.6972 Rec: 0.9252 F1: 0.7952
TP: 99.0 TN: 3413.0 FP: 43.0 FN: 8.0

Epoch 18/24
----------
train Loss: 0.0104 Acc: 0.9977 Pre: 1.0000 Rec: 0.9157 F1: 0.9560
TP: 76.0 TN: 2943.0 FP: 0.0 FN: 7.0
val Loss: 0.0167 Acc: 0.9947 Pre: 0.8454 Rec: 0.9880 F1: 0.9111
TP: 82.0 TN: 2928.0 FP: 15.0 FN: 1.0
test Loss: 0.0509 Acc: 0.9851 Pre: 0.6875 Rec: 0.9252 F1: 0.7888
TP: 99.0 TN: 3411.0 FP: 45.0 FN: 8.0

Epoch 19/24
----------
train Loss: 0.0177 Acc: 0.9954 Pre: 1.0000 Rec: 0.8313 F1: 0.9079
TP: 69.0 TN: 2943.0 FP: 0.0 FN: 14.0
val Loss: 0.0230 Acc: 0.9944 Pre: 0.8300 Rec: 1.0000 F1: 0.9071
TP: 83.0 TN: 2926.0 FP: 17.0 FN: 0.0
test Loss: 0.0563 Acc: 0.9834 Pre: 0.6600 Rec: 0.9252 F1: 0.7704
TP: 99.0 TN: 3405.0 FP: 51.0 FN: 8.0

Epoch 20/24
----------
train Loss: 0.0181 Acc: 0.9944 Pre: 0.9714 Rec: 0.8193 F1: 0.8889
TP: 68.0 TN: 2941.0 FP: 2.0 FN: 15.0
val Loss: 0.0201 Acc: 0.9947 Pre: 0.8384 Rec: 1.0000 F1: 0.9121
TP: 83.0 TN: 2927.0 FP: 16.0 FN: 0.0
test Loss: 0.0597 Acc: 0.9829 Pre: 0.6513 Rec: 0.9252 F1: 0.7645
TP: 99.0 TN: 3403.0 FP: 53.0 FN: 8.0

Epoch 21/24
----------
train Loss: 0.0236 Acc: 0.9941 Pre: 0.9577 Rec: 0.8193 F1: 0.8831
TP: 68.0 TN: 2940.0 FP: 3.0 FN: 15.0
val Loss: 0.0147 Acc: 0.9954 Pre: 0.8632 Rec: 0.9880 F1: 0.9213
TP: 82.0 TN: 2930.0 FP: 13.0 FN: 1.0
test Loss: 0.0475 Acc: 0.9860 Pre: 0.7021 Rec: 0.9252 F1: 0.7984
TP: 99.0 TN: 3414.0 FP: 42.0 FN: 8.0

Epoch 22/24
----------
train Loss: 0.0217 Acc: 0.9960 Pre: 1.0000 Rec: 0.8554 F1: 0.9221
TP: 71.0 TN: 2943.0 FP: 0.0 FN: 12.0
val Loss: 0.0180 Acc: 0.9947 Pre: 0.8384 Rec: 1.0000 F1: 0.9121
TP: 83.0 TN: 2927.0 FP: 16.0 FN: 0.0
test Loss: 0.0500 Acc: 0.9854 Pre: 0.6923 Rec: 0.9252 F1: 0.7920
TP: 99.0 TN: 3412.0 FP: 44.0 FN: 8.0

Epoch 23/24
----------
train Loss: 0.0129 Acc: 0.9967 Pre: 0.9740 Rec: 0.9036 F1: 0.9375
TP: 75.0 TN: 2941.0 FP: 2.0 FN: 8.0
val Loss: 0.0210 Acc: 0.9950 Pre: 0.8469 Rec: 1.0000 F1: 0.9171
TP: 83.0 TN: 2928.0 FP: 15.0 FN: 0.0
test Loss: 0.0572 Acc: 0.9840 Pre: 0.6689 Rec: 0.9252 F1: 0.7765
TP: 99.0 TN: 3407.0 FP: 49.0 FN: 8.0

Epoch 24/24
----------
train Loss: 0.0144 Acc: 0.9967 Pre: 0.9867 Rec: 0.8916 F1: 0.9367
TP: 74.0 TN: 2942.0 FP: 1.0 FN: 9.0
val Loss: 0.0096 Acc: 0.9967 Pre: 0.9011 Rec: 0.9880 F1: 0.9425
TP: 82.0 TN: 2934.0 FP: 9.0 FN: 1.0
test Loss: 0.0398 Acc: 0.9877 Pre: 0.7442 Rec: 0.8972 F1: 0.8136
TP: 96.0 TN: 3423.0 FP: 33.0 FN: 11.0

Training complete in 28m 48s
val_train val test
Epoch 0/24
----------
train Loss: 0.0974 Acc: 0.9729 Pre: 0.5455 Rec: 0.0723 F1: 0.1277
TP: 6.0 TN: 2938.0 FP: 5.0 FN: 77.0
val Loss: 0.0496 Acc: 0.9884 Pre: 0.7353 Rec: 0.9036 F1: 0.8108
TP: 75.0 TN: 2916.0 FP: 27.0 FN: 8.0
test Loss: 0.0613 Acc: 0.9801 Pre: 0.6364 Rec: 0.7850 F1: 0.7029
TP: 84.0 TN: 3408.0 FP: 48.0 FN: 23.0

Epoch 1/24
----------
train Loss: 0.0646 Acc: 0.9812 Pre: 0.7955 Rec: 0.4217 F1: 0.5512
TP: 35.0 TN: 2934.0 FP: 9.0 FN: 48.0
val Loss: 0.0311 Acc: 0.9917 Pre: 0.8295 Rec: 0.8795 F1: 0.8538
TP: 73.0 TN: 2928.0 FP: 15.0 FN: 10.0
test Loss: 0.0734 Acc: 0.9753 Pre: 0.6610 Rec: 0.3645 F1: 0.4699
TP: 39.0 TN: 3436.0 FP: 20.0 FN: 68.0

Epoch 2/24
----------
train Loss: 0.0429 Acc: 0.9898 Pre: 0.9194 Rec: 0.6867 F1: 0.7862
TP: 57.0 TN: 2938.0 FP: 5.0 FN: 26.0
val Loss: 0.0336 Acc: 0.9881 Pre: 0.7117 Rec: 0.9518 F1: 0.8144
TP: 79.0 TN: 2911.0 FP: 32.0 FN: 4.0
test Loss: 0.0490 Acc: 0.9832 Pre: 0.6556 Rec: 0.9252 F1: 0.7674
TP: 99.0 TN: 3404.0 FP: 52.0 FN: 8.0

Epoch 3/24
----------
train Loss: 0.0372 Acc: 0.9907 Pre: 0.9104 Rec: 0.7349 F1: 0.8133
TP: 61.0 TN: 2937.0 FP: 6.0 FN: 22.0
val Loss: 0.0463 Acc: 0.9812 Pre: 0.5956 Rec: 0.9759 F1: 0.7397
TP: 81.0 TN: 2888.0 FP: 55.0 FN: 2.0
test Loss: 0.0609 Acc: 0.9790 Pre: 0.5964 Rec: 0.9252 F1: 0.7253
TP: 99.0 TN: 3389.0 FP: 67.0 FN: 8.0

Epoch 4/24
----------
train Loss: 0.0398 Acc: 0.9894 Pre: 0.8696 Rec: 0.7229 F1: 0.7895
TP: 60.0 TN: 2934.0 FP: 9.0 FN: 23.0
val Loss: 0.0152 Acc: 0.9964 Pre: 0.9286 Rec: 0.9398 F1: 0.9341
TP: 78.0 TN: 2937.0 FP: 6.0 FN: 5.0
test Loss: 0.0311 Acc: 0.9913 Pre: 0.8167 Rec: 0.9159 F1: 0.8634
TP: 98.0 TN: 3434.0 FP: 22.0 FN: 9.0

Epoch 5/24
----------
train Loss: 0.0307 Acc: 0.9907 Pre: 0.8873 Rec: 0.7590 F1: 0.8182
TP: 63.0 TN: 2935.0 FP: 8.0 FN: 20.0
val Loss: 0.0120 Acc: 0.9960 Pre: 0.8989 Rec: 0.9639 F1: 0.9302
TP: 80.0 TN: 2934.0 FP: 9.0 FN: 3.0
test Loss: 0.0293 Acc: 0.9913 Pre: 0.8115 Rec: 0.9252 F1: 0.8646
TP: 99.0 TN: 3433.0 FP: 23.0 FN: 8.0

Epoch 6/24
----------
train Loss: 0.0309 Acc: 0.9924 Pre: 0.9688 Rec: 0.7470 F1: 0.8435
TP: 62.0 TN: 2941.0 FP: 2.0 FN: 21.0
val Loss: 0.0199 Acc: 0.9947 Pre: 0.8602 Rec: 0.9639 F1: 0.9091
TP: 80.0 TN: 2930.0 FP: 13.0 FN: 3.0
test Loss: 0.0394 Acc: 0.9862 Pre: 0.7071 Rec: 0.9252 F1: 0.8016
TP: 99.0 TN: 3415.0 FP: 41.0 FN: 8.0

Epoch 7/24
----------
train Loss: 0.0228 Acc: 0.9941 Pre: 0.9851 Rec: 0.7952 F1: 0.8800
TP: 66.0 TN: 2942.0 FP: 1.0 FN: 17.0
val Loss: 0.0233 Acc: 0.9934 Pre: 0.8119 Rec: 0.9880 F1: 0.8913
TP: 82.0 TN: 2924.0 FP: 19.0 FN: 1.0
test Loss: 0.0427 Acc: 0.9851 Pre: 0.6849 Rec: 0.9346 F1: 0.7905
TP: 100.0 TN: 3410.0 FP: 46.0 FN: 7.0

Epoch 8/24
----------
train Loss: 0.0266 Acc: 0.9927 Pre: 0.9420 Rec: 0.7831 F1: 0.8553
TP: 65.0 TN: 2939.0 FP: 4.0 FN: 18.0
val Loss: 0.0204 Acc: 0.9941 Pre: 0.8283 Rec: 0.9880 F1: 0.9011
TP: 82.0 TN: 2926.0 FP: 17.0 FN: 1.0
test Loss: 0.0430 Acc: 0.9862 Pre: 0.7042 Rec: 0.9346 F1: 0.8032
TP: 100.0 TN: 3414.0 FP: 42.0 FN: 7.0

Epoch 9/24
----------
train Loss: 0.0220 Acc: 0.9937 Pre: 0.9444 Rec: 0.8193 F1: 0.8774
TP: 68.0 TN: 2939.0 FP: 4.0 FN: 15.0
val Loss: 0.0201 Acc: 0.9934 Pre: 0.8119 Rec: 0.9880 F1: 0.8913
TP: 82.0 TN: 2924.0 FP: 19.0 FN: 1.0
test Loss: 0.0445 Acc: 0.9846 Pre: 0.6781 Rec: 0.9252 F1: 0.7826
TP: 99.0 TN: 3409.0 FP: 47.0 FN: 8.0

Epoch 10/24
----------
train Loss: 0.0171 Acc: 0.9950 Pre: 0.9474 Rec: 0.8675 F1: 0.9057
TP: 72.0 TN: 2939.0 FP: 4.0 FN: 11.0
val Loss: 0.0180 Acc: 0.9937 Pre: 0.8200 Rec: 0.9880 F1: 0.8962
TP: 82.0 TN: 2925.0 FP: 18.0 FN: 1.0
test Loss: 0.0429 Acc: 0.9851 Pre: 0.6875 Rec: 0.9252 F1: 0.7888
TP: 99.0 TN: 3411.0 FP: 45.0 FN: 8.0

Epoch 11/24
----------
train Loss: 0.0209 Acc: 0.9947 Pre: 1.0000 Rec: 0.8072 F1: 0.8933
TP: 67.0 TN: 2943.0 FP: 0.0 FN: 16.0
val Loss: 0.0176 Acc: 0.9960 Pre: 0.8737 Rec: 1.0000 F1: 0.9326
TP: 83.0 TN: 2931.0 FP: 12.0 FN: 0.0
test Loss: 0.0450 Acc: 0.9860 Pre: 0.7021 Rec: 0.9252 F1: 0.7984
TP: 99.0 TN: 3414.0 FP: 42.0 FN: 8.0

Epoch 12/24
----------
train Loss: 0.0181 Acc: 0.9947 Pre: 0.9855 Rec: 0.8193 F1: 0.8947
TP: 68.0 TN: 2942.0 FP: 1.0 FN: 15.0
val Loss: 0.0241 Acc: 0.9934 Pre: 0.8058 Rec: 1.0000 F1: 0.8925
TP: 83.0 TN: 2923.0 FP: 20.0 FN: 0.0
test Loss: 0.0501 Acc: 0.9840 Pre: 0.6689 Rec: 0.9252 F1: 0.7765
TP: 99.0 TN: 3407.0 FP: 49.0 FN: 8.0

Epoch 13/24
----------
train Loss: 0.0172 Acc: 0.9950 Pre: 0.9722 Rec: 0.8434 F1: 0.9032
TP: 70.0 TN: 2941.0 FP: 2.0 FN: 13.0
val Loss: 0.0149 Acc: 0.9950 Pre: 0.8469 Rec: 1.0000 F1: 0.9171
TP: 83.0 TN: 2928.0 FP: 15.0 FN: 0.0
test Loss: 0.0428 Acc: 0.9865 Pre: 0.7122 Rec: 0.9252 F1: 0.8049
TP: 99.0 TN: 3416.0 FP: 40.0 FN: 8.0

Epoch 14/24
----------
train Loss: 0.0243 Acc: 0.9944 Pre: 0.9853 Rec: 0.8072 F1: 0.8874
TP: 67.0 TN: 2942.0 FP: 1.0 FN: 16.0
val Loss: 0.0145 Acc: 0.9957 Pre: 0.8646 Rec: 1.0000 F1: 0.9274
TP: 83.0 TN: 2930.0 FP: 13.0 FN: 0.0
test Loss: 0.0411 Acc: 0.9868 Pre: 0.7174 Rec: 0.9252 F1: 0.8082
TP: 99.0 TN: 3417.0 FP: 39.0 FN: 8.0

Epoch 15/24
----------
train Loss: 0.0161 Acc: 0.9960 Pre: 0.9494 Rec: 0.9036 F1: 0.9259
TP: 75.0 TN: 2939.0 FP: 4.0 FN: 8.0
val Loss: 0.0123 Acc: 0.9974 Pre: 0.9121 Rec: 1.0000 F1: 0.9540
TP: 83.0 TN: 2935.0 FP: 8.0 FN: 0.0
test Loss: 0.0379 Acc: 0.9865 Pre: 0.7122 Rec: 0.9252 F1: 0.8049
TP: 99.0 TN: 3416.0 FP: 40.0 FN: 8.0

Epoch 16/24
----------
train Loss: 0.0132 Acc: 0.9960 Pre: 0.9610 Rec: 0.8916 F1: 0.9250
TP: 74.0 TN: 2940.0 FP: 3.0 FN: 9.0
val Loss: 0.0183 Acc: 0.9950 Pre: 0.8469 Rec: 1.0000 F1: 0.9171
TP: 83.0 TN: 2928.0 FP: 15.0 FN: 0.0
test Loss: 0.0455 Acc: 0.9862 Pre: 0.7071 Rec: 0.9252 F1: 0.8016
TP: 99.0 TN: 3415.0 FP: 41.0 FN: 8.0

Epoch 17/24
----------
train Loss: 0.0120 Acc: 0.9954 Pre: 0.9481 Rec: 0.8795 F1: 0.9125
TP: 73.0 TN: 2939.0 FP: 4.0 FN: 10.0
val Loss: 0.0244 Acc: 0.9924 Pre: 0.7830 Rec: 1.0000 F1: 0.8783
TP: 83.0 TN: 2920.0 FP: 23.0 FN: 0.0
test Loss: 0.0517 Acc: 0.9843 Pre: 0.6711 Rec: 0.9346 F1: 0.7812
TP: 100.0 TN: 3407.0 FP: 49.0 FN: 7.0

Epoch 18/24
----------
train Loss: 0.0128 Acc: 0.9960 Pre: 0.9733 Rec: 0.8795 F1: 0.9241
TP: 73.0 TN: 2941.0 FP: 2.0 FN: 10.0
val Loss: 0.0176 Acc: 0.9957 Pre: 0.8646 Rec: 1.0000 F1: 0.9274
TP: 83.0 TN: 2930.0 FP: 13.0 FN: 0.0
test Loss: 0.0454 Acc: 0.9854 Pre: 0.6923 Rec: 0.9252 F1: 0.7920
TP: 99.0 TN: 3412.0 FP: 44.0 FN: 8.0

Epoch 19/24
----------
train Loss: 0.0242 Acc: 0.9927 Pre: 0.9692 Rec: 0.7590 F1: 0.8514
TP: 63.0 TN: 2941.0 FP: 2.0 FN: 20.0
val Loss: 0.0177 Acc: 0.9960 Pre: 0.8737 Rec: 1.0000 F1: 0.9326
TP: 83.0 TN: 2931.0 FP: 12.0 FN: 0.0
test Loss: 0.0443 Acc: 0.9860 Pre: 0.7021 Rec: 0.9252 F1: 0.7984
TP: 99.0 TN: 3414.0 FP: 42.0 FN: 8.0

Epoch 20/24
----------
train Loss: 0.0167 Acc: 0.9957 Pre: 1.0000 Rec: 0.8434 F1: 0.9150
TP: 70.0 TN: 2943.0 FP: 0.0 FN: 13.0
val Loss: 0.0197 Acc: 0.9947 Pre: 0.8384 Rec: 1.0000 F1: 0.9121
TP: 83.0 TN: 2927.0 FP: 16.0 FN: 0.0
test Loss: 0.0491 Acc: 0.9843 Pre: 0.6735 Rec: 0.9252 F1: 0.7795
TP: 99.0 TN: 3408.0 FP: 48.0 FN: 8.0

Epoch 21/24
----------
train Loss: 0.0126 Acc: 0.9964 Pre: 0.9865 Rec: 0.8795 F1: 0.9299
TP: 73.0 TN: 2942.0 FP: 1.0 FN: 10.0
val Loss: 0.0141 Acc: 0.9974 Pre: 0.9121 Rec: 1.0000 F1: 0.9540
TP: 83.0 TN: 2935.0 FP: 8.0 FN: 0.0
test Loss: 0.0396 Acc: 0.9865 Pre: 0.7122 Rec: 0.9252 F1: 0.8049
TP: 99.0 TN: 3416.0 FP: 40.0 FN: 8.0

Epoch 22/24
----------
train Loss: 0.0156 Acc: 0.9934 Pre: 0.8889 Rec: 0.8675 F1: 0.8780
TP: 72.0 TN: 2934.0 FP: 9.0 FN: 11.0
val Loss: 0.0097 Acc: 0.9980 Pre: 0.9425 Rec: 0.9880 F1: 0.9647
TP: 82.0 TN: 2938.0 FP: 5.0 FN: 1.0
test Loss: 0.0374 Acc: 0.9885 Pre: 0.7500 Rec: 0.9252 F1: 0.8285
TP: 99.0 TN: 3423.0 FP: 33.0 FN: 8.0

Epoch 23/24
----------
train Loss: 0.0166 Acc: 0.9947 Pre: 0.9855 Rec: 0.8193 F1: 0.8947
TP: 68.0 TN: 2942.0 FP: 1.0 FN: 15.0
val Loss: 0.0099 Acc: 0.9983 Pre: 0.9432 Rec: 1.0000 F1: 0.9708
TP: 83.0 TN: 2938.0 FP: 5.0 FN: 0.0
test Loss: 0.0357 Acc: 0.9877 Pre: 0.7333 Rec: 0.9252 F1: 0.8182
TP: 99.0 TN: 3420.0 FP: 36.0 FN: 8.0

Epoch 24/24
----------
train Loss: 0.0179 Acc: 0.9950 Pre: 0.9474 Rec: 0.8675 F1: 0.9057
TP: 72.0 TN: 2939.0 FP: 4.0 FN: 11.0
val Loss: 0.0154 Acc: 0.9964 Pre: 0.8830 Rec: 1.0000 F1: 0.9379
TP: 83.0 TN: 2932.0 FP: 11.0 FN: 0.0
test Loss: 0.0430 Acc: 0.9862 Pre: 0.7071 Rec: 0.9252 F1: 0.8016
TP: 99.0 TN: 3415.0 FP: 41.0 FN: 8.0

Training complete in 28m 28s
```

# Smooth Results

In [15]:
def pos_negs(predictions, gt):
    correct = np.where(np.array(predictions) == np.array(gt), 1, 0)
    incorrect = np.where(np.array(predictions) == np.array(gt), 0, 1)
    
    tp = np.where(correct * np.where(predictions == np.array(1), 1, 0), 1, 0)
    tn = np.where(correct * np.where(predictions == np.array(0), 1, 0), 1, 0)
    fp = np.where(incorrect * np.where(predictions == np.array(1), 1, 0), 1, 0)
    fn = np.where(incorrect * np.where(predictions == np.array(0), 1, 0), 1, 0)
    
    return tp, tn, fp, fn

def acc_prf1(predictions, gt):
    tp, tn, fp, fn = pos_negs(predictions, gt)
    
    acc = (np.sum(tp) + np.sum(tn)) / (np.sum(tp) + np.sum(tn) + np.sum(fp) + np.sum(fn))
    precision = np.sum(tp) / (np.sum(tp) + np.sum(fp))
    recall = np.sum(tp) / (np.sum(tp) + np.sum(fn))
    f1 = 2 * precision * recall / (precision + recall)
    
    return acc, precision, recall, f1, np.sum(tp), np.sum(tn), np.sum(fp), np.sum(fn)

def smooth_predictions(preds, window_radius = 3):
    result = []
    for i in range(len(preds)):
        start = max(0, i - window_radius)
        end = min(len(preds), i + window_radius)
        window = preds[start:end]
        result += [max(window, key=window.count)]
    
    return result

In [16]:
path = 'models/transfer_learning_tutorial_correct_train'

log_file_img = open(os.path.join(
    path, 'test_results_image_classifier.log'), 'w')
log_file_smoothed = open(os.path.join(
    path, 'test_results_smoothed.log'), 'w')

for seed in range(5):
    print(seed)
    model_ts = models.resnet50(pretrained=True)
    num_ftrs = model_ts.fc.in_features
    model_ts.fc = nn.Linear(num_ftrs, 1)
    model_ts.load_state_dict(torch.load(
        os.path.join(path, 'seed_{}.pth'.format(seed))
    ))
    model_ts.to(device)
    criterion = nn.BCEWithLogitsLoss()
    
    model = model_ts.eval()   # Set model to evaluate mode
    dl = dataloaders['test']
    dataset_size = dataset_sizes['test']
    
    running_corrects = 0
    true_positives = 0.
    true_negatives = 0.
    false_positives = 0.
    false_negatives = 0.

    i = 0

    predictions = []
    gt_labels = []
    # Iterate over data.
    for inputs, labels in dl:
        inputs = inputs.to(device)
        labels = labels.to(device).float()

        # forward
        # track history if only in train
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            preds = torch.where(
                outputs >= 0.,
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
            target = torch.where(
                labels >= 0.5,
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device)
            )

        predictions += preds.cpu().numpy().tolist()
        gt_labels += labels.cpu().numpy().tolist()

        # statistics
        label_vals = torch.where(
            labels >= 0.5,
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device)
        )
        correct = preds.view(label_vals.shape) == label_vals.data
        running_corrects += torch.sum(correct)

        true_positives += torch.sum(
            torch.where(
                (correct == 1.) * (label_vals == 1.),
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
        )
        true_negatives += torch.sum(
            torch.where(
                (correct == 1.) * (label_vals == 0.),
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
        )
        false_positives += torch.sum(
            torch.where(
                (correct == 0.) * (label_vals == 0.),
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
        )
        false_negatives += torch.sum(
            torch.where(
                (correct == 0.) * (label_vals == 1.),
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
        )

        num_fp = torch.sum(
        torch.where(
            (correct == 0.) * (label_vals == 0.),
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
        )

    #     if num_fp > 0:
    #         print(num_fp)
    #         print(torch.where(
    #             (correct == 0.) * (label_vals == 0.),
    #             torch.tensor([1.]).to(device),
    #             torch.tensor([0.]).to(device)))
    #         print(i)
    #         out = torchvision.utils.make_grid(inputs)
    # #         imshow(out, title=preds.cpu().numpy().tolist())
    #         imshow(out)

        i += 1

    epoch_acc = running_corrects.double() / dataset_size
    epoch_pre = safe_divide(true_positives, (true_positives + false_positives))
    epoch_recall = safe_divide(true_positives, (true_positives + false_negatives))
    epoch_f1 = safe_divide(2 * epoch_pre * epoch_recall, (epoch_pre + epoch_recall))

    print('Acc: {:.4f} Pre: {:.4f} Rec: {:.4f} F1: {:.4f}'.format(
        epoch_acc, epoch_pre, epoch_recall, epoch_f1))
    print('TP: {} TN: {} FP: {} FN: {}'.format(
        true_positives.data, true_negatives.data, false_positives.data, 
        false_negatives.data))
    
    log_file_img.write('Seed: {0}\t'
                   'Acc: {acc:.4f}\t'
                   'Pre: {pre:.4f}\t'
                   'Rec: {rec:.4f}\t'
                   'F1: {f1:.4f}\t'
                   'TP: {tp} '
                   'TN: {tn} '
                   'FP: {fp} '
                   'FN: {fn}\n'.format(
                       seed,
                       acc=epoch_acc, pre=epoch_pre, rec=epoch_recall,
                       f1=epoch_f1, tp=int(true_positives.data),
                       tn=int(true_negatives.data),
                       fp=int(false_positives.data), fn=int(false_negatives.data)
                   ))
    log_file_img.flush()

    predictions = [p[0] for p in predictions]

    smoothed_preds = smooth_predictions(predictions, 3)

    print("Smoothed stats:")
    print(acc_prf1(smoothed_preds, gt_labels))
    
    sm_acc, sm_pre, sm_rec, sm_f1, sm_tp, sm_tn, sm_fp, sm_fn = acc_prf1(
        smoothed_preds, gt_labels)
    
    log_file_smoothed.write('Seed: {0}\t'
                   'Acc: {acc:.4f}\t'
                   'Pre: {pre:.4f}\t'
                   'Rec: {rec:.4f}\t'
                   'F1: {f1:.4f}\t'
                   'TP: {tp} '
                   'TN: {tn} '
                   'FP: {fp} '
                   'FN: {fn}\n'.format(
                       seed,
                       acc=sm_acc, pre=sm_pre, rec=sm_rec,
                       f1=sm_f1, tp=sm_tp,
                       tn=sm_tn,
                       fp=sm_fp, fn=sm_fn
                   ))
    log_file_smoothed.flush()

0
Acc: 0.9896 Pre: 0.7734 Rec: 0.9252 F1: 0.8426
TP: 99.0 TN: 3427.0 FP: 29.0 FN: 8.0
Smoothed stats:
(0.9935447656469267, 0.8818181818181818, 0.9065420560747663, 0.8940092165898617, 97, 3443, 13, 10)
1
Acc: 0.9829 Pre: 0.6667 Rec: 0.8598 F1: 0.7510
TP: 92.0 TN: 3410.0 FP: 46.0 FN: 15.0
Smoothed stats:
(0.9904574796519787, 0.8288288288288288, 0.8598130841121495, 0.8440366972477064, 92, 3437, 19, 15)
2
Acc: 0.9862 Pre: 0.7266 Rec: 0.8692 F1: 0.7915
TP: 93.0 TN: 3421.0 FP: 35.0 FN: 14.0
Smoothed stats:
(0.9941060903732809, 0.9056603773584906, 0.897196261682243, 0.9014084507042254, 96, 3446, 10, 11)
3
Acc: 0.9877 Pre: 0.7442 Rec: 0.8972 F1: 0.8136
TP: 96.0 TN: 3423.0 FP: 33.0 FN: 11.0
Smoothed stats:
(0.9915801291046871, 0.8407079646017699, 0.8878504672897196, 0.8636363636363636, 95, 3438, 18, 12)
4
Acc: 0.9862 Pre: 0.7071 Rec: 0.9252 F1: 0.8016
TP: 99.0 TN: 3415.0 FP: 41.0 FN: 8.0
Smoothed stats:
(0.9912994667415099, 0.8220338983050848, 0.9065420560747663, 0.8622222222222222, 97, 3435, 2